In [ ]:
from scipy import *
from numpy import *
import bqplot.pyplot as plt
from bqplot.interacts import BrushSelector
import ipywidgets as widgets
from IPython.display import display, Latex, HTML, Math
from scipy.optimize import fsolve
from scipy.misc import derivative
from tokenize import TokenError
import sympy as sym
from sympy.parsing.sympy_parser import parse_expr

## The Newton-Raphson Method

The Newton-Raphson Method is an iterative method for numerically determining the root of a function.  It depends on being able to evaluate the function at a point and being able to evaluate the derivative of the function at the point.

If the function in question is $f(x)$, with first derivative $f'(x)$, begin with an initial guess for the location of the root of the function.  Usually, this guess should be motivated by looking at a plot of the function, and guessing something in the neighborhood of where the function appears to cross the horizontal axis.  Let's call our initial guess $x_0$.  Clearly, we can find both the value of the function at $x_0$, $f(x_0)$, as well as the slope of the line tangent to the function at $x_0$, namely $f'(x_0)$.  The idea behind the method is to follow this tangent line at $x_0$ to the point where it (the *tangent line*) crosses the horizontal axis, and use that point as our new, improved guess (which we will call $x_1$) for the root of the original function.  But we know that 
$$\text{slope} = \frac{\text{rise}}{\text{run}},$$
so if we think of the "run" as the distance from our initial guess, $x_0$, to our next guess, $x_1$, and the "rise" as the distance from $0$ (the definition of the location of $x_1$) and $f(x_0)$ (the tangent point of the line), then
$$\text{slope} = \frac{(0 - f(x_0))}{(x_1 - x_0)} = f'(x_0)$$
Solving this for $x_1$, we get:
$$x_1 = x_0 - \frac{f(x_0)}{f'(x_0)}.$$

Now we just 'iterate' this process, which means that we consider $x_1$ now to be our guess and calculate a new, better guess, labelled $x_2$, from it using the value of the function and its derivative at $x_1$:
$$x_2 = x_1 - \frac{f(x_1)}{f'(x_1)}.$$

Repeat until the new, improved guess gets as close as you like to the value of the true root.  How do you know how close it is?  If the most recent estimate is $x_n$, then the difference $|x_n - x_{n-1}|$ is an estimate of the uncertainty in the location of the root.  If you need to get an estimate that is better than $10^{-6}$, for example, continue the iterations until the absolute value of the difference between the last estimate and the next-to-last estimate is smaller than $10^{-6}$.


#### Note:
The Newton-Raphson Method is *not* guaranteed to converge!  It could happen that the iterations take the estimate further and further away from the true root!  It is *crucial* that the initial guess is in the vicinity of the true root -- a "good" initial guess is necessary for convergence.  With a good initial point, the method is very quick.

For a function that has multiple roots, the initial guess will determine which root is found by the method.

## Newton-Raphson Interactive Explorer

You can use the interactive plotting tool below to follow along with the iterations described above.

Starting with the default function, $e^{2-x}-3$, which is plotted in blue.  The red dot on the horizontal axis is the location of the true zero; this is what we are trying to locate with the method.  The initial guess is plotted as a green box on the horizontal axis.  Follow the dotted green line up to the function itself, which is at a height of $f(x_0)$.  We then calculate the slope of the tangent line at that point (*i.e.*, we evaluate $f'(x_0)$).  That determines the magenta tangent line, which we then follow down to its intersection with the horizontal axis, marked with a magenta diamond.  This is our new guess for the location of th root.

Clicking on the green **iterate** button will change the magenta diamond into a green square, indicating that $x_1$ is now our "initial guess", and repeat the calculations of the green dotted line, the magenta dashed line, and the new and improved estimate, marked again with a magenta diamond (and its value reported in the magenta box, along with the number of iterations).

If the green and magenta marks get too compressed to see easily visually, you can use the mouse to select a new, zoomed area for the plot.  If you zoom badly, or just want to start over, click the **reset limits** button.

You can enter your own functions, change the initial guess, and change the domain on which the function is plotted.

In [ ]:
def set_function(strFunc):
    try:
        local_func = lambda x: eval(strFunc)
        f = parse_expr(strFunc)
    except (TokenError, ValueError, SyntaxError):
        print('malformed function')
        return None
    return local_func, f

In [211]:
def func(x):
    return exp(-(x-2)) - 3

# initial function
strFunc = "exp(-(x-2)) - 3"
func, fstr = set_function(strFunc)
sym_func = sym.latex(fstr)

# layouts
fig_layout = widgets.Layout(height = "500px", width = "800px")
title_layout = widgets.Layout(display='flex', justify_content='center', width = "70%", 
                              align_items = 'center', border = 'none')
fig_margin_dict = dict(top=30, bottom = 30, left = 20, right = 10)

# title label and formatting
title_format = "<h2><font color='blue'>$f(x) = {}$</font></h2>"
title_label = widgets.HTMLMath(title_format.format(sym_func),
                              layout = title_layout)

# Main figure
main_fig = plt.figure( layout = fig_layout, fig_margin = fig_margin_dict)
nPts = 200
x = linspace(-3,3,nPts)

# to control what appears in legends, use "display_legend = True" on items to show and do NOT 
# call plt.legend()

function_curve = plt.plot(x, func(x), 'b-', display_legend = True, labels = ['function'], stroke_width = 3)
x0 = fsolve(func, 0)
root = plt.plot(x0, func(x0), colors = ['red'], marker = 'circle', labels = ['true root'])

# initial guess plots
xcur = -2.0
start_pt = plt.plot([xcur], [0], colors=['green'], marker = 'square', display_legend = True,
                labels = ['initial guess'])
rise = plt.plot([xcur, xcur], [0, func(xcur)], line_style = "dotted", colors = ['green'])
fprime = derivative(func, xcur, dx = 1e-6)
b = func(xcur) - fprime*xcur

# name function for clarity
def line_func(x):
    return fprime*x + b
line = plt.plot(x, line_func(x), colors = ['magenta'], line_style = "dashed", display_legend = False)
xcur = -b / fprime
end_pt = plt.plot([xcur], [0], colors = ['magenta'], marker = 'diamond', display_legend = True,
              labels = ['new estimate'])

# function entry
func_label = widgets.Label("Enter function: ")
func_entry = widgets.Text(strFunc, continuous_update = False)
# domain entry
dom_layout = widgets.Layout(width = '75px')
dom_label = widgets.Label("Enter domain: ")
dom_low = widgets.FloatText(value = -3.0, step = 0.1, layout = dom_layout)
dom_high = widgets.FloatText(value = 3.0, step = 0.1, layout = dom_layout)
func_box = widgets.HBox([func_label, func_entry, dom_label, dom_low, dom_high])

# iterations box
xguess = -2
iter_count  = 0
init_label = widgets.Label("Enter initial guess: ")
guess = widgets.FloatText(value = -2.0, step = 0.1, layout = dom_layout)
btn_iter = widgets.Button(button_style = 'success', description = "iterate")
#iter_form = "<b>{}</b>"
iter_form = "{}"
lab_layout = widgets.Layout( width = '175px')
iter_disc = widgets.Label("# iterations = ", layout = lab_layout)
iter_out = widgets.Label(value = iter_form.format(0), description = "# iterations", style=dict(font_size="24px",
                                                                                              font_weight='bold'))
btn_reset = widgets.Button(description = "reset limits", disabled = True)
iter_box = widgets.HBox([init_label, guess, btn_iter, iter_disc, iter_out, btn_reset], grid_gap = "10px")

# output
out_layout = widgets.Layout(border="blue solid 2px")
out_str = "<div style='padding:2%'> The true zero is <b><font color = 'red'>{:.4f}</font></b> </div> "
output = widgets.HTML(value = out_str.format(x0[0]), layout = out_layout)
cur_layout = widgets.Layout(border="magenta solid 2px")
cur_str = "<p style='padding:2%'>  The estimate for iteration {} is <b><font color = 'magenta'>{:.4f}</font></b> </p> "
cur_widg = widgets.HTML(value = cur_str.format(iter_count, xguess), layout = cur_layout)
out_box_layout = widgets.Layout(width = '75%', grid_gap = "10px")
out_box = widgets.HBox([output, cur_widg], layout = out_box_layout)



def iterate(stuff):
    global  xcur
    iter_count = int(iter_out.value)
    iter_count += 1
    iter_out.value = str(iter_count)
    start_pt.x = [xcur]
    rise.x = [xcur, xcur]
    rise.y = [0, func(xcur)]
    fprime = derivative(func, xcur, dx = 1e-6)
    b = func(xcur) - fprime*xcur
    # grab the correct set of x values:
    x = linspace(dom_low.value, dom_high.value, nPts)
    def line_func(x):
        return fprime*x + b
    line.y = line_func(x)
    xcur = -b / fprime
    end_pt.x = [xcur]
    cur_widg.value = cur_str.format(iter_count, xcur)
    
xcur = xguess
def start(stuff):
    global xcur
    iter_count = 0
    iter_out.value = str(iter_count)
    xcur = guess.value
    x0 = fsolve(func, xcur)
    output.value = out_str.format(x0[0])
    root.x = x0
    iterate('yark')

def update_plot(change):
    global func
    strFunc = func_entry.value
    func, fstr = set_function(strFunc)
    title_label.value = title_format.format(sym.latex(fstr))
    low = dom_low.value
    high = dom_high.value
    xlimits.min = low
    xlimits.max = high
    x = linspace(dom_low.value, dom_high.value, nPts)
    y = func(x)
    # fix y-axis limits:
    ylimits.min = min(y)
    ylimits.max = max(y)
    
    # update main plot
    function_curve.x = x
    line.x = x
    function_curve.y = y
    start('yark')
    
def brush_callback(change):
    brushing = br_sel.brushing
    if not brushing:
        sel = br_sel.selected
        xlimits.min = sel[0][0]
        ylimits.min = sel[0][1]
        xlimits.max = sel[1][0]
        ylimits.max = sel[1][1]
        btn_reset.disabled = False
    br_sel.selected = None

def reset_limits(stuff):
    update_plot("reset")
    
ylimits = plt.ylim(-50, 150)
xlimits = plt.xlim(-3, 3)
plt.hline(0, colors = ['peru'])

br_sel = BrushSelector(x_scale = function_curve.scales['x'], 
                       y_scale = function_curve.scales['y'], marks = [function_curve], color='lightblue')
main_fig.interaction = br_sel
br_sel.observe(brush_callback, names = ['brushing'])
# connections
btn_iter.on_click(iterate)
btn_reset.on_click(reset_limits)
guess.observe(start, names = 'value')
func_entry.observe(update_plot, names = 'value')
dom_low.observe(update_plot, names = 'value')
dom_high.observe(update_plot, names = 'value')

widgets.VBox([title_label, main_fig, func_box,  iter_box, out_box])


In [ ]:
%%html
<style>
div.input{
    display:none;
}
</style>

In [191]:
iter_disc.config(font=("Courier", 20))

AttributeError: 'Label' object has no attribute 'config'

In [183]:
dir(iter_out)

['__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_notifiers',
 '_call_widget_constructed',
 '_comm_changed',
 '_compare',
 '_cross_validation_lock',
 '_default_keys',
 '_display_callbacks',
 '_dom_classes',
 '_gen_repr_from_keys',
 '_get_embed_state',
 '_get_trait_default_generator',
 '_handle_custom_msg',
 '_handle_displayed',
 '_handle_msg',
 '_holding_sync',
 '_ipython_display_',
 '_is_numpy',
 '_lock_property',
 '_log_default',
 '_model_id',
 '_model_module',
 '_model_module_version',
 '_model_name',
 '_msg_callbacks',
 '_notify_observers',
 '_notify_trait',
 '_property_lock',
 '_register_validator',
 '_remove_notif

In [203]:
s = int(iter_out.value.strip())
s

0